## Ejercicio

Vamos a usar el siguiente dataset 

<https://www.kaggle.com/c/competitive-data-science-predict-future-sales>. 

El problema es predecir las ventas diarias de las últimas dos semanas de Noviembre de 2015

In [1]:
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
plt.rcParams['figure.figsize'] = [20, 10]
plt.rc('xtick', labelsize=20)     
plt.rc('ytick', labelsize=20)

### Preparación de datos

In [15]:
import datetime
sales = pd.read_csv("./data/sales_train.csv")
sales['date'] = sales['date'].apply(lambda x:datetime.datetime.strptime(x, '%d.%m.%Y'))

In [16]:
total_daily_item_sales = sales.groupby(['date', 'item_id'], as_index=False)[['item_cnt_day']].sum()

In [18]:
total_monthly_item_sales = total_daily_item_sales\
    .groupby([pd.Grouper(key='date', freq="M"), pd.Grouper(key='item_id')])\
    [["item_cnt_day"]].sum().reset_index().rename({"item_cnt_day": "item_cnt_month"}, axis=1)
total_monthly_item_sales.shape

(233912, 3)

In [19]:
dates = total_monthly_item_sales['date'].unique()
item_ids = total_monthly_item_sales['item_id'].unique()

dates_item_ids = [[date, item_id] for date in dates for item_id in item_ids]
complete_df = pd.DataFrame(dates_item_ids, columns=["date", "item_id"])
complete_df.shape

(741438, 2)

In [20]:
complete_df = complete_df.merge(total_monthly_item_sales, how="left").fillna(0)

In [21]:
complete_df.head(20)

,date,item_id,item_cnt_month
0,2013-01-31,32,299.0
1,2013-01-31,33,61.0
2,2013-01-31,53,8.0
3,2013-01-31,59,18.0
4,2013-01-31,85,17.0
5,2013-01-31,98,361.0
6,2013-01-31,133,7.0
7,2013-01-31,154,28.0
8,2013-01-31,155,22.0
9,2013-01-31,354,6.0


In [22]:
def add_lag(df, lag, col_name='item_cnt_month'):
    df[col_name + "-" + str(lag)] = df.groupby(['item_id'])[col_name].shift(lag)
    return df

def add_lags(df, lags, col_name='item_cnt_month'):
    for lag in lags:
        df = add_lag(df, lag, col_name)
    return df

complete_df = add_lags(complete_df, range(1, 13))

In [24]:
complete_df.head()

,date,item_id,item_cnt_month,item_cnt_month-1,item_cnt_month-2,item_cnt_month-3,item_cnt_month-4,item_cnt_month-5,item_cnt_month-6,item_cnt_month-7,item_cnt_month-8,item_cnt_month-9,item_cnt_month-10,item_cnt_month-11,item_cnt_month-12
0,2013-01-31,32,299.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-01-31,33,61.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-01-31,53,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-01-31,59,18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-01-31,85,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
Xy = complete_df.query("date > '2013-12-31'")

def separate_X_y(Xy, sales_col='item_cnt_month'):
    y = Xy[sales_col]
    X = Xy.drop(sales_col, axis=1)
    if 'date' in X.columns:
        X = X.drop('date', axis=1)
    return X, y
        
def train_dev_test_split(Xy):
    X_test, y_test = separate_X_y(Xy.query("date == '2015-10-31'"))
    X_dev, y_dev = separate_X_y(Xy.query("date == '2015-09-30'"))
    X_train, y_train = separate_X_y(Xy.query("date <= '2015-08-31'"))
    return X_train, X_dev, X_test, y_train, y_dev, y_test

X_train, X_dev, X_test, y_train, y_dev, y_test = train_dev_test_split(Xy)

In [27]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
reg = LinearRegression()
reg.fit(X_train.drop("item_id", axis=1), y_train)
y_dev_hat = reg.predict(X_dev.drop("item_id", axis=1))
np.sqrt(mean_squared_error(y_dev, y_dev_hat))

28.360797623711704

In [29]:
import warnings
from sklearn.ensemble import RandomForestRegressor
warnings.filterwarnings("ignore", category=FutureWarning)


reg_rf = RandomForestRegressor(random_state=667)
reg_rf.fit(X_train.drop("item_id", axis=1), y_train)
y_dev_hat = reg_rf.predict(X_dev.drop("item_id", axis=1))
np.sqrt(mean_squared_error(y_dev, y_dev_hat))

/Users/CotePelaez/anaconda/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


29.333380817161505

*Ejercicio*: aquí tenéis un baseline. La idea es mejorarlo incluyendo
otras variables.